# Pandas Data Structures Overview
In this section, we will discuss the `Series`, `Index`, and `DataFrame`. To do so, we will read in a snippet of the CSV file we will work with later. Don't worry about that part yet, though.

## About the Data
In this notebook, we will be working 5 rows from the Earthquake data collected over September 18, 2018 - October 13, 2018 (obtained from the US Geological Survey (USGS) using the [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Working with NumPy Arrays
We will read in a short CSV file (using `numpy`) for some sample data. 

In [2]:
import numpy as np

data = np.genfromtxt('data/example_data.csv', delimiter=';', names=True, dtype=None, encoding='UTF')

data

array([('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1),
       ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0),
       ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0),
       ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0),
       ('2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea', 'mww', 5.6, 'green', 1)],
      dtype=[('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

We can find the dimensions with the `shape` attribute:

In [4]:
data.shape

(5,)

We can find the data types with the `dtype` attribute:

In [5]:
data.dtype

dtype([('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

Each of the entries in the array is a row from the CSV file. NumPy arrays contain a single data type (unlike lists, which allow mixed types); this allows for fast, vectorized options. When we read in the data, we got an array of `numpy.void` objects, which are created to store flexible types. This is because NumPy is storing several different data types per row: 4 strings, a float, and an integer. This means we can't take advantage of the performance improvements NumPy provides for single data type objects. Say we want to find the maximum magnitude, we can use a list comprehension with these flexible objects in order to select the 3rd index of each row. This makes a list, then we can take the maximum:

In [6]:
%%timeit
max([row[3] for row in data])

3.44 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


If we, instead, create a NumPy array for each column, this operation is much easier (and more efficient) to perform. We can make a dictionary where the keys are the column names and the values are NumPy arrays of the data:

In [20]:
array_dict = {}
for i, col in enumerate(data.dtype.names):
    array_dict[col]= np.array([row[i] for row in data])

print(array_dict)
print('\n')
print('Data for comparison \n', data)

{'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
       '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
       '2018-10-12 02:52:03.620'], dtype='<U23'), 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'), 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'), 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]), 'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'), 'tsunami': array([1, 0, 0, 0, 1])}


Data for comparison 
 [('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1)
 ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0)
 ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0)
 ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0)

Grabbing the maximum magnitude is now simply a matter of selecting the `mag` key and calling the `max()` method. This is nearly twice as fast as the list comprehension implementation when dealing with just 5 entries, imagine how much worse the first attempt will perform on large data sets:

In [22]:
%%timeit
array_dict['mag'].max()

1.58 µs ± 70.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
array_dict

{'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'tsunami': array([1, 0, 0, 0, 1])}

However, this representation has other issues. Say we wanted to grab all the information for the earthquake with the maximum magnitude, how would we go about that? We would need to find the index of the maximum and then for each of the keys in the dictionary grab that index:

In [23]:
np.array([value[array_dict['mag'].argmax()] for key, value in array_dict.items()])

array(['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
       'mww', '6.7', 'green', '1'], dtype='<U32')

The result is now a NumPy array of strings (our numeric values were converted), and we are now in the format from earlier. Also, consider trying to sort the data by magnitude from smallest to largest. In the first representation, we would have to sort the rows by examining the 3rd index. With the second representation, we would have to determine the order for the indices from the `mag` column, and then sort all the other arrays with those same indices. Clearly, working with several NumPy arrays of different data types at once is a bit cumbersome. 

## `Series`
The `Series` is a 1D NumPy array with some additional functionality.

In [24]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')

place

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

### Getting the dimensions of the series
Just as with NumPy, we can use `shape` to get the dimensions in as `(rows, columns)`. `Series` objects are a single column, so they only have values for the rows dimension. 

In [25]:
place.shape

(5,)

### Getting the data type
A `Series` object holds a single data type. Here it is `'O'` for object.

In [26]:
place.dtype

dtype('O')

### Getting the name of the series
The NumPy array held the `name` in the `dtype` attribute; here, we can access it directly: 

In [27]:
place.name

'place'

### Isolating the values from the series (as an array)
The `Series` is storing its values as a NumPy array:

In [28]:
place.values

array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype=object)

## `Index`
The addition of the `Index` makes the `Series` more powerful than the array. We can get the index from the `index` attribute of the `Series`:

In [29]:
place_index = place.index
place_index

RangeIndex(start=0, stop=5, step=1)

The `Index` is also built on top of a NumPy array:

In [30]:
place_index.values

array([0, 1, 2, 3, 4], dtype=int64)

We can check its data type, just like with a `Series` object:

In [31]:
place_index.dtype

dtype('int64')

Same for the dimensions:

In [32]:
place_index.shape

(5,)

We can check if the values are unique:

In [33]:
place_index.is_unique

True

With NumPy we can perform arithmetic operations element-wise between arrays:

In [34]:
place_index.values

array([0, 1, 2, 3, 4], dtype=int64)

Pandas supports this as well, and the index determines how element-wise operations are performed. With addition, only the matching indices are summed:

In [37]:
pd.Series(np.linspace(0,10, num=5))+ pd.Series(np.linspace(0,10,num=5),index = pd.Index([1,2,3,4,5]))

0     NaN
1     2.5
2     7.5
3    12.5
4    17.5
5     NaN
dtype: float64

We aren't limited to the integer indices of list-like structures, and we can label our rows. The labels can be altered at any time and be things like dates or even another column. In [chapter 3](https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/tree/master/ch_03), we will discuss how to perform some operations on the `Index` in order to change it. Then, in [chapter 4](https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/tree/master/ch_04), we will use the `Index` for operations merging data and aggregating it.

## `DataFrame`
Having a `Series` object for each column is an improvement over the numpy representation; however, we still have the same problem when wanting to sort based on a value or grab an entire row out. The `DataFrame` gives us a representation of a table formed from many `Series` objects that form the columns and a shared `Index` object that labels the rows. We can create a `DataFrame` object from either of the numpy representations we were working with earlier (we could make a `Series` object for each column, but there is no need to do so):

In [39]:
##df = pd.DataFrame(array_dict) 

# this will also work with the first representation
df = pd.DataFrame(data)

df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


### Isolating the values of a dataframe as a NumPy ndarray (n-dimensional array)

In [40]:
df.values

array([['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
        'mww', 6.7, 'green', 1],
       ['2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww',
        5.2, 'green', 0],
       ['2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww',
        5.7, 'green', 0],
       ['2018-10-12 21:09:49.240',
        '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0],
       ['2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea',
        'mww', 5.6, 'green', 1]], dtype=object)

### Getting the data types

In [41]:
df.dtypes

time        object
place       object
magType     object
mag        float64
alert       object
tsunami      int32
dtype: object

### Getting the dimensions of the dataframe
As (rows, columns).

In [42]:
df.shape

(5, 6)

### Getting the index of the dataframe

In [44]:
df.index

RangeIndex(start=0, stop=5, step=1)

### Getting the columns in the dataframe
Notice that the columns are actually an `Index` just on a different axis (columns are the horizontal index while rows are the vertical index).

In [45]:
df.columns

Index(['time', 'place', 'magType', 'mag', 'alert', 'tsunami'], dtype='object')

Note that we can also perform arithmetic on dataframes, but the results may seem a little strange. Pandas will only perform the operation when both the index and column match. Here, since `+` with strings means concatenation, pandas concatenated the `time`, `place`, `magType`, and `alert` columns across dataframes. The `mag` and `tsunami` columns got summed:

In [46]:
df + df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.5602018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia262km NW of Oze...",mwwmww,13.4,greengreen,2
1,2018-10-13 04:34:15.5802018-10-13 04:34:15.580,"25km E of Bitung, Indonesia25km E of Bitung, I...",mwwmww,10.4,greengreen,0
2,2018-10-13 00:13:46.2202018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu42km WNW of Sola, Van...",mwwmww,11.4,greengreen,0
3,2018-10-12 21:09:49.2402018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala13km E of...",mwwmww,11.4,greengreen,0
4,2018-10-12 02:52:03.6202018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea128km SE of...",mwwmww,11.2,greengreen,2


In [48]:
mydf = pd.DataFrame([[5,12,18,42],[16,28,36,2]], index= list('cd'), columns= list('mnop'))

mydf

,m,n,o,p
c,5,12,18,42
d,16,28,36,2


In [49]:
mydf.dtypes

m    int64
n    int64
o    int64
p    int64
dtype: object

In [51]:
import pandas as pd
days = pd.Series([420, 380, 390], index=['day1', 'day2', 'day3'])
days[0]

C:\Users\Kabey\AppData\Local\Temp\ipykernel_33860\881945012.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  days[0]


420

In [54]:
calorie_data = pd.DataFrame({'day': ['day1','day2','day3','day4','day5']
                           ,'calories': [450, 300, 345, 520, 600]
                           ,'duration_min': [30, 25, 29, 39, 48]
                           })
calorie_data

calorie_data.loc[0]

day             day1
calories         450
duration_min      30
Name: 0, dtype: object

In [56]:
np.random.seed(0)

pd.Series(np.random.rand(5),name='random')

0    0.548814
1    0.715189
2    0.602763
3    0.544883
4    0.423655
Name: random, dtype: float64